<a href="https://colab.research.google.com/github/anicelysantos/book-python-para-analise-de-dados/blob/main/series_temporais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Estudos do livro "Python para análise de dados" cap. 11*

# **Imports**

In [121]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from dateutil.parser import parse
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
import pytz
from pandas.tseries.offsets import Hour

# **Tipos de dados e ferramentas para data e hora**

In [ ]:
now = datetime.now()
now

datetime.datetime(2021, 9, 23, 13, 29, 13, 172509)

In [ ]:
#now.year, now.month, now.day
now.day, now.month, now.year

(23, 9, 2021)

In [ ]:
#import lá em cima timedelta
#timedelta mostra a diferença de tempo entre dois objetos datetime

delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(days=926, seconds=56700)

In [ ]:
delta.days

926

In [ ]:
delta.seconds

56700

In [ ]:
start = datetime(2011,1,7)
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [ ]:
start - 2 * timedelta(12)


datetime.datetime(2010, 12, 14, 0, 0)

**Conversão entre string e datetime**

In [ ]:
stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [ ]:
#stamp.strftime('%Y-%m-%d')
stamp.strftime('%F') #mesma coisa de cima

'2011-01-03'

In [ ]:
value = '2011-01-03'

In [ ]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [ ]:
datestrs = ['9/19/2021','9/20/2020']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2021, 9, 19, 0, 0), datetime.datetime(2020, 9, 20, 0, 0)]

In [ ]:
#Import do parse lá no inicio
parse('21-09-2021')

datetime.datetime(2021, 9, 21, 0, 0)

In [ ]:
parse('Sep 21, 2021 09:02 AM')

datetime.datetime(2021, 9, 21, 9, 2)

In [ ]:
#Dia antes do mês, padrão Brasil
parse('21/09/2021', dayfirst=True)

datetime.datetime(2021, 9, 21, 0, 0)

In [ ]:
datastrs = ['2021-09-21 09:04:00', '2021-09-21 09:05:00']
pd.to_datetime(datastrs)

DatetimeIndex(['2021-09-21 09:04:00', '2021-09-21 09:05:00'], dtype='datetime64[ns]', freq=None)

In [ ]:
#Lidar com dados ausentes (None)
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2021-09-19', '2020-09-20', 'NaT'], dtype='datetime64[ns]', freq=None)

In [ ]:
#NaT é o valor nulo do pandas para tempo/data (dados de timestamp)
idx[2]

NaT

In [ ]:
pd.isnull(idx)

array([False, False,  True])

# **Básico sobre séries temporais**

In [ ]:
#import datetime no inicio
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5), datetime(2011, 1, 7), datetime(2011, 1, 8), datetime(2011,1,10), datetime(2011, 1, 12)]

In [ ]:
ts = pd.Series(np.random.randn(6), index=dates)
ts

2011-01-02   -0.684071
2011-01-05    0.081168
2011-01-07   -1.106606
2011-01-08    1.353748
2011-01-10   -0.530924
2011-01-12    1.887462
dtype: float64

In [ ]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
#[::2]seleciona um elemento a cada dois
ts + ts[::2]

2011-01-02   -1.368142
2011-01-05         NaN
2011-01-07   -2.213212
2011-01-08         NaN
2011-01-10   -1.061847
2011-01-12         NaN
dtype: float64

In [ ]:
ts.index.dtype

dtype('<M8[ns]')

In [ ]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

**Indexação, seleção e geração de subconjuntos**

In [ ]:
stamp = ts.index[2]
ts[stamp]

-1.1066061780998508

In [ ]:
#Passar uma string para ser interpretada como data
ts['1/10/2011']

-0.530923652479331

In [ ]:
ts['20110110']

-0.530923652479331

In [ ]:
#series temporais mais longas podem selecionar o ano ou só o ano e o mês para trazer trechos do dataframe
longer_ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
longer_ts

2000-01-01    0.199748
2000-01-02    1.260583
2000-01-03   -1.892064
2000-01-04   -0.280157
2000-01-05   -0.182719
                ...   
2002-09-22    0.550165
2002-09-23    0.557361
2002-09-24   -0.392346
2002-09-25    1.958250
2002-09-26    2.116483
Freq: D, Length: 1000, dtype: float64

In [ ]:
longer_ts['2001']

2001-01-01   -0.335982
2001-01-02   -1.338210
2001-01-03   -0.267116
2001-01-04   -1.065923
2001-01-05   -1.497908
                ...   
2001-12-27    0.084817
2001-12-28   -0.619236
2001-12-29   -1.090974
2001-12-30   -0.163828
2001-12-31   -0.728719
Freq: D, Length: 365, dtype: float64

In [ ]:
longer_ts['2001-05']

2001-05-01    1.869002
2001-05-02    1.467071
2001-05-03   -0.628286
2001-05-04    0.713497
2001-05-05    0.394756
2001-05-06   -1.709258
2001-05-07   -1.214505
2001-05-08    2.366634
2001-05-09    0.007984
2001-05-10    1.860136
2001-05-11   -0.550236
2001-05-12   -0.559171
2001-05-13    0.442912
2001-05-14    1.268620
2001-05-15   -0.735149
2001-05-16    1.167073
2001-05-17   -2.038632
2001-05-18    0.348276
2001-05-19    1.997680
2001-05-20    0.059078
2001-05-21    0.012315
2001-05-22    0.592081
2001-05-23   -1.111719
2001-05-24   -0.298991
2001-05-25   -0.008758
2001-05-26    0.727762
2001-05-27    0.788274
2001-05-28    0.182244
2001-05-29    0.249627
2001-05-30    3.162677
2001-05-31   -0.207069
Freq: D, dtype: float64

In [ ]:
ts[datetime(2011, 1, 7):]

2011-01-07   -1.106606
2011-01-08    1.353748
2011-01-10   -0.530924
2011-01-12    1.887462
dtype: float64

In [ ]:
ts

2011-01-02   -0.684071
2011-01-05    0.081168
2011-01-07   -1.106606
2011-01-08    1.353748
2011-01-10   -0.530924
2011-01-12    1.887462
dtype: float64

In [ ]:
ts['1/6/2011':'1/11/2011']

2011-01-07   -1.106606
2011-01-08    1.353748
2011-01-10   -0.530924
dtype: float64

In [ ]:
#Fatiar um periodo entre duas datas
ts.truncate(after='1/9/2011')

2011-01-02   -0.684071
2011-01-05    0.081168
2011-01-07   -1.106606
2011-01-08    1.353748
dtype: float64

In [ ]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100,4), index=dates, columns=['Colorado', 'Texas', 'New York', 'Ohio'])

In [ ]:
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,-0.451923,-0.733515,0.279948,-0.959145
2001-05-09,-0.599619,-0.071136,0.608583,1.035421
2001-05-16,1.280251,0.185831,-0.967143,-1.470001
2001-05-23,1.103447,-0.385917,-0.197072,0.208652
2001-05-30,1.021483,1.050951,0.723737,1.067807


**Séries temporais com índices duplicados**

In [ ]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000', '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)

In [ ]:
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [ ]:
dup_ts.index.is_unique

False

In [ ]:
dup_ts['1/3/2000']

4

In [ ]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

In [ ]:
#agregar os dados únicos
grouped = dup_ts.groupby(level=0)
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [ ]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

# **Intervalos de datas, frequências e deslocamentos**

In [ ]:
ts

2011-01-02   -0.684071
2011-01-05    0.081168
2011-01-07   -1.106606
2011-01-08    1.353748
2011-01-10   -0.530924
2011-01-12    1.887462
dtype: float64

In [ ]:
#String 'D' é o equivalente a frequência diária
resampler = ts.resample('D')


**Gerando intervalos de datas**

In [ ]:
#pandas.date_range gera index com tamanho especifico
index = pd.date_range('2012-04-01', '2012-06-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [ ]:
#Se passar só a data do inicio ou do fim, precisa passar o periodo
pd.date_range(start='2021-08-01', periods=20)

DatetimeIndex(['2021-08-01', '2021-08-02', '2021-08-03', '2021-08-04',
               '2021-08-05', '2021-08-06', '2021-08-07', '2021-08-08',
               '2021-08-09', '2021-08-10', '2021-08-11', '2021-08-12',
               '2021-08-13', '2021-08-14', '2021-08-15', '2021-08-16',
               '2021-08-17', '2021-08-18', '2021-08-19', '2021-08-20'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
pd.date_range(end='2021-09-01', periods=20)

DatetimeIndex(['2021-08-13', '2021-08-14', '2021-08-15', '2021-08-16',
               '2021-08-17', '2021-08-18', '2021-08-19', '2021-08-20',
               '2021-08-21', '2021-08-22', '2021-08-23', '2021-08-24',
               '2021-08-25', '2021-08-26', '2021-08-27', '2021-08-28',
               '2021-08-29', '2021-08-30', '2021-08-31', '2021-09-01'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
#'BM' trás um trecho de data contendo o ultimo dia útil de cada mês
pd.date_range('2020-01-01','2021-01-01', freq='BM')

DatetimeIndex(['2020-01-31', '2020-02-28', '2020-03-31', '2020-04-30',
               '2020-05-29', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-30', '2020-11-30', '2020-12-31'],
              dtype='datetime64[ns]', freq='BM')

In [ ]:
pd.date_range('2021-07-03 12:56:31', periods=5)

DatetimeIndex(['2021-07-03 12:56:31', '2021-07-04 12:56:31',
               '2021-07-05 12:56:31', '2021-07-06 12:56:31',
               '2021-07-07 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
#normalizar horários com normalize
pd.date_range('2021-08-02 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2021-08-02', '2021-08-03', '2021-08-04', '2021-08-05',
               '2021-08-06'],
              dtype='datetime64[ns]', freq='D')

**Frequências e offset de datas**

In [ ]:
#Import Hour e Minute lá em cima
hour = Hour()
hour

<Hour>

In [ ]:
four_hours = Hour(4)
four_hours

<4 * Hours>

In [ ]:
pd.date_range('2000-01-01', '2001-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               ...
               '2001-01-02 08:00:00', '2001-01-02 12:00:00',
               '2001-01-02 16:00:00', '2001-01-02 20:00:00',
               '2001-01-03 00:00:00', '2001-01-03 04:00:00',
               '2001-01-03 08:00:00', '2001-01-03 12:00:00',
               '2001-01-03 16:00:00', '2001-01-03 20:00:00'],
              dtype='datetime64[ns]', length=2214, freq='4H')

In [ ]:
Hour(2) + Minute(30)

<150 * Minutes>

In [ ]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

**Datas com a semana do mês**

In [ ]:
rng = pd.date_range('2020-01-01', '2020-09-01', freq='WOM-3FRI')
list(rng)

[Timestamp('2020-01-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-02-21 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-03-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-04-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-05-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-06-19 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-07-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2020-08-21 00:00:00', freq='WOM-3FRI')]

**Deslocamento de datas (adiantando e atrasando)**

In [ ]:
ts = pd.Series(np.random.randn(4), index=pd.date_range('1/1/2020', periods=4, freq='M'))
ts

2020-01-31    1.654308
2020-02-29    0.704188
2020-03-31   -1.012582
2020-04-30   -0.807274
Freq: M, dtype: float64

In [ ]:
ts.shift(2)

2020-01-31         NaN
2020-02-29         NaN
2020-03-31    1.654308
2020-04-30    0.704188
Freq: M, dtype: float64

In [ ]:
ts.shift(-2)

2020-01-31   -1.012582
2020-02-29   -0.807274
2020-03-31         NaN
2020-04-30         NaN
Freq: M, dtype: float64

In [ ]:
ts/ts.shift(1) - 1

2020-01-31         NaN
2020-02-29   -0.574331
2020-03-31   -2.437942
2020-04-30   -0.202756
Freq: M, dtype: float64

In [ ]:
ts.shift(2,freq='M')

2020-03-31    1.654308
2020-04-30    0.704188
2020-05-31   -1.012582
2020-06-30   -0.807274
Freq: M, dtype: float64

In [ ]:
ts.shift(3, freq='D')

2020-02-03    1.654308
2020-03-03    0.704188
2020-04-03   -1.012582
2020-05-03   -0.807274
dtype: float64

In [ ]:
ts.shift(1, freq='90T') #T é minutos

2020-01-31 01:30:00    1.654308
2020-02-29 01:30:00    0.704188
2020-03-31 01:30:00   -1.012582
2020-04-30 01:30:00   -0.807274
dtype: float64

**Deslocando datas com offsets**

In [ ]:
#Import Day e MonthEnd lá em cima
now = datetime(2011, 11, 17)
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

In [ ]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [ ]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

In [ ]:
offset = MonthEnd()

In [ ]:
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [ ]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

In [ ]:
ts = pd.Series(np.random.randn(20), index=pd.date_range('1/15/2000', periods=20, freq='4d'))
ts

2000-01-15   -0.094457
2000-01-19    1.800217
2000-01-23   -0.735293
2000-01-27   -2.209752
2000-01-31    0.867398
2000-02-04    1.254965
2000-02-08    1.004025
2000-02-12    0.246823
2000-02-16   -0.492568
2000-02-20    0.580840
2000-02-24   -0.946012
2000-02-28   -1.538934
2000-03-03   -0.317310
2000-03-07   -0.068492
2000-03-11   -0.688859
2000-03-15    0.724098
2000-03-19    0.338351
2000-03-23   -0.454508
2000-03-27    0.807206
2000-03-31    0.969676
Freq: 4D, dtype: float64

In [ ]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.074377
2000-02-29    0.015591
2000-03-31    0.163770
dtype: float64

In [ ]:
ts.resample('M').mean()

2000-01-31   -0.074377
2000-02-29    0.015591
2000-03-31    0.163770
Freq: M, dtype: float64

# **Tratamento de fusos horários**

In [ ]:
#Import pytz lá em cima
#Encontrar o nome dos fusohorários
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [ ]:
#Obter um objeto de fuso horário
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

**Localização e conversão dos fuso horários**

In [ ]:
#Series temporais não consideram fuso horario por padrão
rng = pd.date_range('3/9/2012 09:30', periods =6, freq='D')
ts=pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-09 09:30:00    0.308918
2012-03-10 09:30:00   -0.822430
2012-03-11 09:30:00    0.630512
2012-03-12 09:30:00   -0.390624
2012-03-13 09:30:00    0.900467
2012-03-14 09:30:00   -0.006649
Freq: D, dtype: float64

In [ ]:
#Campo tz no índice é None
print(ts.index.tz)

None


In [ ]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [ ]:
ts

2012-03-09 09:30:00    0.308918
2012-03-10 09:30:00   -0.822430
2012-03-11 09:30:00    0.630512
2012-03-12 09:30:00   -0.390624
2012-03-13 09:30:00    0.900467
2012-03-14 09:30:00   -0.006649
Freq: D, dtype: float64

In [ ]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00    0.308918
2012-03-10 09:30:00+00:00   -0.822430
2012-03-11 09:30:00+00:00    0.630512
2012-03-12 09:30:00+00:00   -0.390624
2012-03-13 09:30:00+00:00    0.900467
2012-03-14 09:30:00+00:00   -0.006649
Freq: D, dtype: float64

In [ ]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [ ]:
#É possivel converter uma serie em outro fuso horário
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00    0.308918
2012-03-10 04:30:00-05:00   -0.822430
2012-03-11 05:30:00-04:00    0.630512
2012-03-12 05:30:00-04:00   -0.390624
2012-03-13 05:30:00-04:00    0.900467
2012-03-14 05:30:00-04:00   -0.006649
Freq: D, dtype: float64

In [ ]:
#Mudando o fuso horário para o de Berlim

ts_eastern = ts.tz_localize('America/New_York')
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00    0.308918
2012-03-10 14:30:00+00:00   -0.822430
2012-03-11 13:30:00+00:00    0.630512
2012-03-12 13:30:00+00:00   -0.390624
2012-03-13 13:30:00+00:00    0.900467
2012-03-14 13:30:00+00:00   -0.006649
dtype: float64

In [ ]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00    0.308918
2012-03-10 15:30:00+01:00   -0.822430
2012-03-11 14:30:00+01:00    0.630512
2012-03-12 14:30:00+01:00   -0.390624
2012-03-13 14:30:00+01:00    0.900467
2012-03-14 14:30:00+01:00   -0.006649
dtype: float64

In [ ]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq=None)

**Operações com objetos Timestamp que consideram fuso horários**

In [113]:
Stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-01-06 19:00:00-0500', tz='America/New_York')

In [116]:
#É possivel passar um fuso horário quando se cria um timestamp
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

In [117]:
#Timestampo armazena valor UTC em nanosegundos
stamp_utc.value

1294358400000000000

In [118]:
stamp_utc.tz_convert('America/New_York').value

1294358400000000000

In [119]:
#Offset respeita horário de verão, import Hour lá em cima
#30 minutos antes de iniciar o horário de verão
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [120]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [123]:
#30 minutos antes de sair do horário de verão
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [124]:
stamp + 2* Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

**Operação entre fusos horários diferentes**

In [125]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-07 09:30:00   -0.810006
2012-03-08 09:30:00   -0.879523
2012-03-09 09:30:00   -1.347900
2012-03-12 09:30:00    1.270644
2012-03-13 09:30:00    0.739971
2012-03-14 09:30:00   -0.066495
2012-03-15 09:30:00    1.316009
2012-03-16 09:30:00    0.201217
2012-03-19 09:30:00   -0.106845
2012-03-20 09:30:00   -1.226962
Freq: B, dtype: float64

In [127]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 + ts2
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

# **Período e aritmética com períodos**